In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!apt-get install -y build-essential

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!pip install scikit-surprise --no-cache-dir

In [ ]:
!pip install "numpy<2"

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk

In [ ]:
from surprise import KNNBasic
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split, GridSearchCV
from surprise import accuracy

In [ ]:
data =Dataset.load_builtin('ml-100k', prompt=False)

Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
X_train, X_test = train_test_split(data, test_size=0.2)

In [ ]:
knn_algo = KNNBasic()
knn_algo.fit(X_train)
predicts= knn_algo.test(X_test)
accuracy.rmse(predicts)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9764


0.9763722618034664

In [ ]:
sim_matrix =knn_algo.sim

[[1.         0.5        0.42857143 ... 0.         0.28571429 0.28571429]
 [0.5        1.         0.47887324 ... 0.18518519 1.         0.5       ]
 [0.42857143 0.47887324 1.         ... 0.22222222 0.5        0.2       ]
 ...
 [0.         0.18518519 0.22222222 ... 1.         0.         0.        ]
 [0.28571429 1.         0.5        ... 0.         1.         0.5       ]
 [0.28571429 0.5        0.2        ... 0.         0.5        1.        ]]


2. Train KNN with data use to load_from_file function by Custorm Dataset

In [ ]:
df_ratings = pd.read_csv('/content/drive/MyDrive/Project_AIL303_GROUP2/ml-32m/final_dataset.csv')
df_100k = df_ratings.head(100000)
reader = Reader(
    line_format = 'user item rating',
    sep=',',
    skip_lines= 1,
    rating_scale= (0.5,5))
movies_data = Dataset.load_from_df(
    df_100k[['userId', 'movieId', 'rating']],
    reader= reader)
# trainset, testset = train_test_split(movies_data, test_size=0.2, random_state=42)
print(f"Total {trainset.n_users} users and {trainset.n_items} movies in the trainingset")


Total 1304 users and 7415 movies in the trainingset


In [ ]:
X_train, X_test = train_test_split(movies_data, test_size=0.2, random_state= 42)
sim_options = {'name':'cosine', 'user_based':True}
knnbasic = KNNBasic(sim_options=sim_options)
knnbasic.fit(X_train)
predicts= knnbasic.test(X_test)
accuracy.rmse(predicts)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9024


0.9023937709609376

2.2 Chạy với Grid , tìm ra hyper tối ưu

In [ ]:
param_grid = {
    'k': [5,10,20,30, 40],
    'sim_options': {
        'name': ['cosine'],
        'user_based': [True]
}
    }
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=5, n_jobs=-1)
gs.fit(movies_data)
best_k = gs.best_params['rmse']['k']
best_sim_option_name = gs.best_params['rmse']['sim_options']['name']
best_sim_option_user_based = gs.best_params['rmse']['sim_options']['user_based']
algo = KNNBasic(k=best_k, sim_options={'name': best_sim_option_name, 'user_based': best_sim_option_user_based})
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.6844


0.6844439046474681

In [ ]:
print(f"Best k: {best_k}")
print(f"Best sim_option_name: {best_sim_option_name}")
print(f"Best sim_option_user_based: {best_sim_option_user_based}")

Best k: 5
Best sim_option_name: cosine
Best sim_option_user_based: True


In [ ]:
knnbasic.sim

array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])